In [1]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

#試取資料
file = open("./log/(1).pickle", "rb")
data = pickle.load(file)
file.close()
type(data['ml'])

dict

In [2]:
game_info = data["ml"]["scene_info"]
game_command = data["ml"]["command"]
print(game_info)
print(game_command)

[{'frame': 0, 'status': 'GAME_ALIVE', 'snake_head': (40, 40), 'snake_body': [(40, 30), (40, 20), (40, 10)], 'food': (30, 250)}, {'frame': 1, 'status': 'GAME_ALIVE', 'snake_head': (40, 50), 'snake_body': [(40, 40), (40, 30), (40, 20)], 'food': (30, 250)}, {'frame': 2, 'status': 'GAME_ALIVE', 'snake_head': (30, 50), 'snake_body': [(40, 50), (40, 40), (40, 30)], 'food': (30, 250)}, {'frame': 3, 'status': 'GAME_ALIVE', 'snake_head': (30, 60), 'snake_body': [(30, 50), (40, 50), (40, 40)], 'food': (30, 250)}, {'frame': 4, 'status': 'GAME_ALIVE', 'snake_head': (30, 70), 'snake_body': [(30, 60), (30, 50), (40, 50)], 'food': (30, 250)}, {'frame': 5, 'status': 'GAME_ALIVE', 'snake_head': (30, 80), 'snake_body': [(30, 70), (30, 60), (30, 50)], 'food': (30, 250)}, {'frame': 6, 'status': 'GAME_ALIVE', 'snake_head': (30, 90), 'snake_body': [(30, 80), (30, 70), (30, 60)], 'food': (30, 250)}, {'frame': 7, 'status': 'GAME_ALIVE', 'snake_head': (30, 100), 'snake_body': [(30, 90), (30, 80), (30, 70)], 'f

In [3]:
for i in range(2,181): #到155可以
    path = "./log/(" + str(i) + ").pickle"
    file = open(path, "rb")
    data = pickle.load(file)
    game_info = game_info + data['ml']['scene_info']
    game_command = game_command + data['ml']['command']
    file.close()  

print(len(game_info))
print(len(game_command))

252520
252520


In [4]:
g = game_info[1]

feature = np.array([g["snake_head"][0], g["snake_head"][1], 0,0,0, g["food"][0],g["food"][1],0,0])
print(feature)
print(game_command[1])
if game_command[1] == "UP": game_command[1] = 0
elif game_command[1] == "DOWN": game_command[1] = 1
elif game_command[1] == "LEFT": game_command[1] = 2
elif game_command[1] == "RIGHT": game_command[1] = 3
else: game_command[1] = 4 

[ 40  50   0   0   0  30 250   0   0]
LEFT


In [5]:
for i in range(2, len(game_info) - 1):
    g = game_info[i]
    g_last = game_info[i-1]
    snakehead_x=g["snake_head"][0]
    snakehead_y=g["snake_head"][1]
    x_dir=g['snake_head'][0] - g_last['snake_head'][0]
    y_dir=g['snake_head'][1] - g_last['snake_head'][1]
    food_x=g["food"][0]
    food_y=g["food"][1]
    if x_dir>0 and y_dir == 0:
        direction = 4 
        wall_disx = 300 - g['snake_head'][0]
    elif x_dir<0 and y_dir == 0:
        direction = 3
        wall_disx = g['snake_head'][0]
    elif y_dir>0 and x_dir == 0:
        direction = 2
        wall_disy = 300 - g['snake_head'][1]
    elif y_dir<0 and x_dir == 0:
        direction = 1
        wall_disy = g['snake_head'][1]
        
    if y_dir == 0:
        wall_disy = max(300 - g['snake_head'][1],g['snake_head'][1])
    elif x_dir == 0:
        wall_disx = max(300 - g['snake_head'][0],g['snake_head'][0])
    body = 0
    wall = 0
    #wall            
    #四個角角
    snake_head = g["snake_head"]
    if ((snake_head[0] > 285 and snake_head[1] > 285)or(snake_head[0] > 285 and snake_head[1] < 5) or (snake_head[0] < 5 and snake_head[1] > 285)or(snake_head[0] < 5 and snake_head[1] > 285)):
        #x方向移動
        if direction >= 3:
            if snake_head[0] > 285:
                if snake_head[1] > 285: #右下
                    wall = 1
                if snake_head[1] < 5:#右上
                    wall = 2
            if snake_head[0] < 5:
                if snake_head[1] > 285:#左下
                    wall = 1
                if snake_head[1] < 5:#左上
                    wall = 2
        #y方向移動
        if direction <= 2:
            if snake_head[1] > 285:
                if snake_head[0] > 285: #右下
                    wall = 3
                if snake_head[0] < 5:#左下
                    wall = 4
            if snake_head[1] < 5:
                if snake_head[0] > 285:#右上
                    wall = 3
                if snake_head[0] < 5:#左下
                    wall = 4          
    else:
                #x方向移動
        if direction >= 3:
            if snake_head[0] >= 285:
                if snake_head[1] > 145: #右下
                    wall = 1
                else:#右上
                    wall = 2
            if snake_head[0] < 5:
                if snake_head[1] > 145:#左下
                    wall = 1
                else:#左上
                    wall = 2
                #y方向移動
        if direction <= 2:
            if snake_head[1] >= 285:
                if snake_head[0] > 145: #右下
                    wall = 3
                else:#左下
                    wall = 4
            if snake_head[1] < 5:
                if snake_head[0] > 145:#右上
                    wall = 3
                else:#左上
                    wall = 4


    #是否離body直線距離<=10?body left==3,right==4,up==1,down==2
    for m in range(0,len(g['snake_body'])):
        if direction==3:#left
            if(g['snake_head'][0]==g['snake_body'][m][0]):
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][1]==g['snake_body'][m][1]+10):
                        body = 2
                    elif(g['snake_head'][1]==g['snake_body'][m][1]-10):
                        body = 1
            if (g['snake_head'][0]==g['snake_body'][m][0]+10 and g['snake_head'][1]==g['snake_body'][m][1]): 
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][1]+10==g['snake_body'][m+1][1]):
                        body = 2
                    elif(g['snake_head'][1]-10==g['snake_body'][m+1][1]):
                        body = 1
                        
                    
        elif direction==4:#right 
            if(g['snake_head'][0]==g['snake_body'][m][0]):
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][1]==g['snake_body'][m][1]+10):
                        body = 2
                    elif(g['snake_head'][1]==g['snake_body'][m][1]-10):
                        body = 1
            if (g['snake_head'][0]==g['snake_body'][m][0]-10 and g['snake_head'][1]==g['snake_body'][m][1]): # 
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][1]+10==g['snake_body'][m+1][1]):
                        body = 2
                    elif(g['snake_head'][1]-10==g['snake_body'][m+1][1]):
                        body = 1
                      
                            
        elif direction==1:#up
            if(g['snake_head'][1]==g['snake_body'][m][1]):
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][0]==g['snake_body'][m][0]+10):
                        body = 4
                    elif(g['snake_head'][0]==g['snake_body'][m][0]-10):
                        body = 3
            if (g['snake_head'][1]==g['snake_body'][m][1]+10 and g['snake_head'][0]==g['snake_body'][m][0]): #
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][0]+10==g['snake_body'][m+1][0]):
                        body = 4
                    elif(g['snake_head'][0]-10==g['snake_body'][m+1][0]):
                        body = 3
                        
        elif direction==2:#down
            if(g['snake_head'][1]==g['snake_body'][m][1]):
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][0]==g['snake_body'][m][0]+10):
                        body = 4
                    elif(g['snake_head'][0]==g['snake_body'][m][0]-10):
                        body = 3
            if (g['snake_head'][1]==g['snake_body'][m][1]-10 and g['snake_head'][0]==g['snake_body'][m][0]): # 
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][0]+10==g['snake_body'][m+1][0]):
                        body = 4
                    elif(g['snake_head'][0]-10==g['snake_body'][m+1][0]):
                        body = 3
                        

    
    feature = np.vstack((feature, [g["snake_head"][0], g["snake_head"][1],direction,wall_disx,wall_disy,food_x,food_y,body,wall]))
    if game_command[i] == "UP": game_command[i] = 0
    elif game_command[i] == "DOWN": game_command[i] = 1
    elif game_command[i] == "LEFT": game_command[i] = 2
    elif game_command[i] == "RIGHT": game_command[i] = 3
    else: game_command[i] = 4 
        
answer = np.array(game_command[1:-1])
print(feature)
print(feature.shape)
print(answer)
print(answer.shape)

[[ 40  50   0 ... 250   0   0]
 [ 30  50   3 ... 250   0   0]
 [ 30  60   2 ... 250   0   0]
 ...
 [110 180   4 ... 210   1   0]
 [110 170   1 ... 210   3   0]
 [100 170   3 ... 210   2   0]]
(252518, 9)
[2 1 1 ... 0 2 0]
(252518,)


# KNN


In [71]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

#資料劃分
x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state=9)
#參數區間
param_grid = {'n_neighbors':[1, 2, 3, 4, 5]}
#交叉驗證 
cv = StratifiedShuffleSplit(n_splits=4, test_size=0.3, random_state=12)
grid = GridSearchCV(KNeighborsClassifier(n_neighbors = 5), param_grid, cv=cv, verbose=10, n_jobs=-1) #n_jobs為平行運算的數量
grid.fit(x_train, y_train)
grid_predictions = grid.predict(x_test)

#儲存
file = open('model_KNN.pickle', 'wb')
pickle.dump(grid, file)
file.close()


Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done   8 out of  20 | elapsed:    7.6s remaining:   11.5s
[Parallel(n_jobs=-1)]: Done  11 out of  20 | elapsed:   11.9s remaining:    9.8s
[Parallel(n_jobs=-1)]: Done  14 out of  20 | elapsed:   12.5s remaining:    5.3s
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed:   15.0s remaining:    2.6s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   15.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   15.2s finished


In [72]:
#最佳參數
print(grid.best_params_)
#預測結果
#print(grid_predictions)
#混淆矩陣
print(confusion_matrix(y_test, grid_predictions))
#分類結果
print(classification_report(y_test, grid_predictions))

{'n_neighbors': 5}
[[ 9942   127   549   541     0]
 [   97 10409   540   524     0]
 [  486   446 14590   150     0]
 [  533   518   169 14741     0]
 [    7     8    16    27     6]]
              precision    recall  f1-score   support

           0       0.90      0.89      0.89     11159
           1       0.90      0.90      0.90     11570
           2       0.92      0.93      0.93     15672
           3       0.92      0.92      0.92     15961
           4       1.00      0.09      0.17        64

    accuracy                           0.91     54426
   macro avg       0.93      0.75      0.76     54426
weighted avg       0.91      0.91      0.91     54426



# MLPClassifier

In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state = 7)
MLPC = MLPClassifier(random_state=1, max_iter=300).fit(x_train, y_train)

MLPC.predict_proba(x_test)
MLPC.predict(x_test)
MLPC.score(x_test, y_test)

file = open("model_MLPC.pickle", "wb")
pickle.dump(MLPC, file)
file.close()

In [7]:
print("Accuracy score (validation): {0:.3f}".format(MLPC.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, MLPC.predict(x_test)))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, MLPC.predict(x_test)))

Accuracy score (validation): 0.997
Confusion Matrix for Raandom Forests:
[[15335    13     0     0     0]
 [   54 16287     0     0     0]
 [    0    23 21778     2    19]
 [    2     1    50 22090     2]
 [   13    24    13    11    39]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     15348
           1       1.00      1.00      1.00     16341
           2       1.00      1.00      1.00     21822
           3       1.00      1.00      1.00     22145
           4       0.65      0.39      0.49       100

    accuracy                           1.00     75756
   macro avg       0.93      0.88      0.90     75756
weighted avg       1.00      1.00      1.00     75756



# HistGradientBoostingClassifier (會delay不要用)

In [268]:
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier

x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state = 7)
HGBC = HistGradientBoostingClassifier(random_state=1, max_iter=300).fit(x_train, y_train)

HGBC.predict_proba(x_test)
HGBC.predict(x_test)
HGBC.score(x_test, y_test)

file = open("model_HGBC.pickle", "wb")
pickle.dump(HGBC, file)
file.close()

In [269]:
print("Accuracy score (validation): {0:.3f}".format(HGBC.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, HGBC.predict(x_test)))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, HGBC.predict(x_test)))

Accuracy score (validation): 0.988
Confusion Matrix for Raandom Forests:
[[6812   20   11   14    6]
 [  17 7053    3   19    5]
 [  79   66 9764    2    5]
 [  71   70    5 9476    7]
 [   4    2    2    8   22]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      6863
           1       0.98      0.99      0.99      7097
           2       1.00      0.98      0.99      9916
           3       1.00      0.98      0.99      9629
           4       0.49      0.58      0.53        38

    accuracy                           0.99     33543
   macro avg       0.89      0.91      0.90     33543
weighted avg       0.99      0.99      0.99     33543



# DecisionTree

In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state = 7)
DT = DecisionTreeClassifier(random_state=0).fit(x_train, y_train)

DT.predict_proba(x_test)
DT.predict(x_test)
DT.score(x_test, y_test)

file = open("model_DT.pickle", "wb")
pickle.dump(DT, file)
file.close()

In [11]:
print("Accuracy score (validation): {0:.3f}".format(DT.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, DT.predict(x_test)))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, DT.predict(x_test)))

Accuracy score (validation): 0.989
Confusion Matrix for Raandom Forests:
[[ 9240    80    19     7    14]
 [   72  9742     5     6    10]
 [   38    42 13098    37    12]
 [   32    41    25 12923    11]
 [    7    10    11    13    20]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      9360
           1       0.98      0.99      0.99      9835
           2       1.00      0.99      0.99     13227
           3       1.00      0.99      0.99     13032
           4       0.30      0.33      0.31        61

    accuracy                           0.99     45515
   macro avg       0.85      0.86      0.85     45515
weighted avg       0.99      0.99      0.99     45515



# Extra tree

In [12]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import ExtraTreeClassifier

x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state = 7)
cls = ExtraTreeClassifier(random_state=0)
ET = BaggingClassifier(cls, random_state=0).fit(x_train, y_train)

ET.predict_proba(x_test)
ET.predict(x_test)
ET.score(x_test, y_test)

file = open("model_ET.pickle", "wb")
pickle.dump(ET, file)
file.close()

In [13]:
print("Accuracy score (validation): {0:.3f}".format(ET.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, ET.predict(x_test)))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, ET.predict(x_test)))

Accuracy score (validation): 0.984
Confusion Matrix for Raandom Forests:
[[ 9206    45    69    37     3]
 [   56  9691    57    30     1]
 [   80    94 13043    10     0]
 [   85    83    10 12854     0]
 [    9    13    12    18     9]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      9360
           1       0.98      0.99      0.98      9835
           2       0.99      0.99      0.99     13227
           3       0.99      0.99      0.99     13032
           4       0.69      0.15      0.24        61

    accuracy                           0.98     45515
   macro avg       0.93      0.82      0.84     45515
weighted avg       0.98      0.98      0.98     45515



# SVM.svc(train很久不要用)

In [274]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SVC = make_pipeline(StandardScaler(), SVC(gamma='auto',probability=True))
SVC.fit(x_train, y_train)

SVC.predict_proba(x_test)
SVC.predict(x_test)
SVC.score(x_test, y_test)

file = open("model_SVC.pickle", "wb")
pickle.dump(SVC, file)
file.close()

In [275]:
print("Accuracy score (validation): {0:.3f}".format(SVC.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, SVC.predict(x_test)))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, SVC.predict(x_test)))

Accuracy score (validation): 0.981
Confusion Matrix for Raandom Forests:
[[6650   10   56  147    0]
 [   5 6980   48   64    0]
 [  64   59 9792    1    0]
 [  77   62    1 9489    0]
 [   4   12    6   12    4]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      6863
           1       0.98      0.98      0.98      7097
           2       0.99      0.99      0.99      9916
           3       0.98      0.99      0.98      9629
           4       1.00      0.11      0.19        38

    accuracy                           0.98     33543
   macro avg       0.98      0.81      0.82     33543
weighted avg       0.98      0.98      0.98     33543

